<a href="https://csdms.colorado.edu"><img style="float: center; width: 75%" src="https://raw.githubusercontent.com/csdms/project/main/assets/CSDMS-logo-color-tagline-hor.png"></a>

# Rainfall-Runoff Modeling

One of the principal applications of hydrology is in the forecasting and prediction of flood peaks and runoff volumes due to large rain and snowmelt events. To do this, researchers use models that simulate the stream response to a water-input event of a given magnitude and spatial and temporal distribution on a drainage basin. These models are usually referred to as rainfall-runoff models.

Rainfall-runoff models include empirical models, conceptual models, and physically based models. These models are used to generate design floods or forecast floods from actual storms. A design flood is used in the design of bridges, levees, or floodplain-management plans. Flood forecasting is the process of predicting the occurrence, magnitude, timing, and duration of floods in a specific area. It is usually done via complex hydrologic models that are varying degrees physically based.

In this tutorial, we will learn to write a simple rainfall-runoff model in Python and use it to estimate runoff hydrograph for an example small watershed.

<img src="https://d3i71xaburhd42.cloudfront.net/c6fb610a3f64a90f2a6f8fe7f04fb723ad71c0dc/10-Figure1-1.png" alt="" style="width:60%; height:auto;"/>


## SCS Curve-Number Method
One of the most widely used rainfall-runoff models for routine design purposes is the SCS Curve-Number method. This method was developed by the U.S. **S**oil **C**onservation **S**ervice (now the U.S. Natural Resources Conservation Service, NRCS). In this method, there are three basic computations: 1) estimation of the effective rainfall, 2) estimation of the peak discharge, and 3) estimation of the runoff hydrograph.

### 1) Estimate effective rainfall
Effective rainfall (or excess rainfall) is the amount of precipitation that contributes to runoff, as opposed to the total rainfall which includes both runoff and infiltration. The SCS method estimates the effective rainfall using total rainfall and watershed storage capacity via an empirical relation: 


$$ w_{eff} = \frac{(w - 0.2 * v_{max})^2}{w + 0.8 * v_{max}}$$

- $w_{eff}$ is the effective rainfall (units: inch).
- $w$ is the total rainfall (units: inch).
- $v_{max}$ is the watershed maximum retention capacity, the maximum ability of a watershed to capture and store water from rainfall events (units: inch).

$v_{max}$ could be estimated using curve numbers ($cn$), which are numbers assigned to each [hydrologic soil group](https://engineering.purdue.edu/mapserve/LTHIA7/documentation/hsg.html) under various land use types. You could learn more about the curve number tables at [here](https://www.hec.usace.army.mil/confluence/hmsdocs/hmstrm/cn-tables).

$$ v_{max} = \frac{1000}{cn} - 10 $$

In [ ]:
# estimate effective rainfall
def calculate_effective_rainfall(cn, w):
    """Calculate the effective rainfall using the SCS-CN method.

    Parameters
    ----------
    cn : float
        Curve Number
    w : float
        Total rainfall (units: inch)

    Returns
    -------
    float
        Effective rainfall (units: inch), rounded to 2 decimal places.
    """

    v_max = 1000 / cn - 10
    w_eff = (w - 0.2 * v_max) ** 2 / (w + 0.8 * v_max)

    return round(w_eff, 2)

### 2) Estimate peak discharge
Peak discharge is estimated using effective rainfall, watershed area, and time to peak (the duration of hydrograph rise).

$$ q_{peak} = \frac{484 * w_{eff}* area}{t_{peak}}$$

- $q_{peak}$ is the peak discharge (units: cfs).
- $area$ is the watershed area (units: $mi^2$).
- $t_{peak}$ is time to peak  (units: hr).

</br>
$t_{peak}$ could be estimated with the function below using duration of effective rainfall and the watershed time of concentration.

$$ t_{peak} = 0.5*t_{rain} + 0.6*t_{con}$$


- $t_{rain}$ is the duration of effective rainfall (units: hr).
- $t_{con}$ is the watershed time of concentration (units: hr).

</br>
$t_{con}$ is the time which takes water to travel from the hydraulically most distant part of the contributing area to the outlet. $t_{con}$ could be estimated with functions using watershed characteristics.

$$ t_{con} = 0.128*(\frac{length}{slope^{0.5}})^{0.79}$$


- $length$ is mainstream length (units: km).
- $slope$ is the sine of main-channel slope angle (dimensionless).


In [ ]:
# calculate time to peak
def calculate_time_to_peak(length, slope, t_rain):
    """
    Calculate the time to peak of a hydrograph using the SCS-CN method.

    Parameters
    ----------
    length : float
        mainstream length (units: km).
    slope : float
        sine of main-channel slope angle (dimensionless).
    t_rain : float
        Duration of effective rainfall (units: hr).

    Returns
    -------
    float
        Time to peak (units: hr), rounded to 2 decimal places.
    """

    t_con = 0.128 * (length / slope**0.5) ** 0.79
    t_peak = 0.5 * t_rain + 0.6 * t_con

    return round(t_peak, 2)

In [ ]:
# calculate peak discharge
def calculate_peak_discharge(area, w_eff, t_peak):
    """
    Calculate the peak discharge of a hydrograph using the SCS-CN method.

    Parameters
    ----------
    area : float
        Area of the watershed (units: mi^2).
    w_eff : float
        Effective rainfall (units: inch).
    t_peak : float
        Time to peak (units: hr).

    Returns
    -------
    float
        Peak discharge (units: cfs), rounded to 2 decimal places.
    """

    q_peak = 484 * w_eff * area / t_peak

    return round(q_peak, 2)

### 3) Estimate runoff hydrograph
SCS method utilizes a dimensionless unit hydrograph to generate the runoff hydrograph. This unit hydrograph was developed with the analysis of a large number of hydrographs developed for watersheds over a range of sizes and locations. Let's first have a look at the SCS dimensionless unit hydrograph. 

In [ ]:
# load SCS unit hydrograph
import pandas as pd

unit_hydrograph = pd.read_csv("./data/scs_unit_hydrograph.csv")
print(unit_hydrograph)

In [ ]:
# plot SCS unit hydrograph
import matplotlib.pyplot as plt

unit_hydrograph.plot(x="time_ratio", y="discharge_ratio")
plt.xlabel("Time Ratio (t/t_peak)")
plt.ylabel("Discharge Ratio (q/q_peak)")
plt.title("SCS Dimensionless Unit Hydrograph")

In the dimensionless unit hydrograph, it has a characteristic shape with a rising limb, peak, and recession limb. The SCS method estimates the time to peak and peak discharge values and use them to scale the time and discharge axes as indicated in the unit hydrograph. With this idea, we could define a function to generate the runoff hydrograph.

In [ ]:
# calculate the runoff hydrograph
def calculate_runoff_hydrograph(q_peak, t_peak, unit_hydrograph):
    """
    Calculate the runoff hydrograph using the SCS-CN method.

    Parameters
    ----------
    q_peak : float
        Peak discharge of the hydrograph (units: cfs).
    t_peak : float
        Time to peak of the hydrograph (units: hr).
    unit_hydrograph : pd.DataFrame
        A DataFrame containing the SCS unit hydrograph data.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns 'time' and 'discharge' representing the computed runoff hydrograph.

    """

    hydrograph = pd.DataFrame(columns=["time", "discharge"])
    hydrograph["time"] = unit_hydrograph["time_ratio"] * t_peak
    hydrograph["discharge"] = unit_hydrograph["discharge_ratio"] * q_peak

    return hydrograph

Using all the functions above, we could define a model function which will first do the model calculation and then show model results.

In [ ]:
# create a model function
def scs_curve_number_model(cn, area, length, slope, t_rain, w, unit_hydrograph):
    """
    Model the runoff hydrograph using the SCS-CN method.

    Parameters
    ----------
    cn : float
        Curve number.
    area : float
        Area of the watershed (units: mi^2).
    length : float
        Mainstream length (units: km).
    slope : float
        Sine of main-channel slope angle (dimensionless).
    t_rain : float
        Duration of effective rainfall (units: hr).
    w : float
        Total rainfall (units: inch).
    unit_hydrograph : pd.DataFrame
        A DataFrame containing the SCS unit hydrograph data.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns 'time' and 'discharge' representing the computed runoff hydrograph.
    """

    # model calculation
    w_eff = calculate_effective_rainfall(cn, w)
    t_peak = calculate_time_to_peak(length, slope, t_rain)
    q_peak = calculate_peak_discharge(area, w_eff, t_peak)
    hydrograph = calculate_runoff_hydrograph(q_peak, t_peak, unit_hydrograph)

    # print results
    print(
        f"SCS Model Results:\n"
        f"Effective rainfall: {w_eff} in\n"
        f"Time to peak: {t_peak} hr\n"
        f"Peak discharge: {q_peak} cfs\n"
    )

    # plot hydrograph
    hydrograph.plot(x="time", y="discharge")
    plt.xlabel("Time (hr)")
    plt.ylabel("Discharge (cfs)")
    plt.title("Runoff Hydrograph")

    return hydrograph

## Model Application

### 1) Example
Let's use the SCS model to estimate the runoff hydrograph for a rain event of 4.2 inch in 3.4 hours for antecedent wetness conditions II on a 1.24 $mi^2$ watershed with a mainstream length of 1.35 km, a main-channel slope of 0.08, and the following land-cover characteristics:

| Land Cover Type| Fraction of Total Area | Curve Number |
|----------|----------|----------|
| Forest (soil group B)   | 0.58  | 58 |
| Forest (soil group C)   | 0.12   | 72  |
| Meadow (soil group A) | 0.21   | 30   |
| Meadow (soil group B) | 0.09   | 58   |




In [ ]:
# show model documentation
help(scs_curve_number_model)

In [ ]:
# calculate weighted-average curve number
cn = 0.58 * 58 + 0.12 * 72 + 0.21 * 30 + 0.09 * 58

# define input variables
area = 1.24
length = 1.35
slope = 0.08
t_rain = 3.4
w = 4.2

# run model for condition II
hydrograph_ii = scs_curve_number_model(
    cn, area, length, slope, t_rain, w, unit_hydrograph
)

### 2) Hands-on Exercise

Now please use the model to estimate the runoff hydrograph with antecedent wetness conditions I and III. The table below shows their corresponding soil wetness condition and weighted curve number for the watershed. How does the soil wetness condition impact the runoff hydrograph?
| Condition| Soil Wetness | Curve Number |
|----------|----------|----------|
| I   | Dry but above wilting point | 35 |
| II  | Average  | 54  |
| III | Near saturation   | 72   |

In [ ]:
# run model for condition I

In [ ]:
# run model for condition III

### 3) Bonus Exercise

Please use the SCS model to explore how the watershed characteristics (e.g., watershed area, mainstream length and slope) and various land use types impact the runoff hydrograph. 

In [ ]:
# write your code here

## References
- Dingman, S. L. (2002) Physical Hydrology. 2nd Edition, Waveland Press, Long Grove. Chapter 9.
- Sitterson, J., Chris Knightes, R. Parmar, K. Wolfe, M. Muche, and B. Avant. (2017) An Overview of Rainfall-Runoff Model Types. U.S. Environmental Protection Agency, Washington, DC, EPA/600/R-17/482, https://cfpub.epa.gov/si/si_public_file_download.cfm?p_download_id=533906&Lab=NERL